In [ ]:
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

# 定义系统消息模板
system_template = "You are a helpful assistant that translates {input_language} to {output_language}."
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)

# 定义用户消息模板
human_template = "{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

# 组合系统消息和用户消息成一个聊天提示
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

print(chat_prompt)

In [ ]:
# 引入第三方大模型
import os
from langchain_community.chat_models import ChatTongyi
from dotenv import load_dotenv
load_dotenv()
chat  = ChatTongyi(model='qwen-plus',
                   top_p=0.9,
                   temperature=0.9,
                   api_key=os.getenv("DASHSCOPE_API_KEY"))
print(chat.invoke('你是谁？'))


# 格式化聊天提示并获取模型响应
response = chat(chat_prompt.format_prompt(
    input_language="English",
    output_language="Chinese",
    text="Who i am."
).to_messages())

print(response)


In [ ]:
from langchain.prompts import PromptTemplate
template = """
作为一个品牌宣传专家，请为以下品类进行宣传：
{% if mood == "手机" %}
请为以下产品的手机提供一个宣传方案{product}!
{% elif mood == "电脑" %}
请为以下产品的电脑提供一个宣传方案{product}!
{% else %}
请为以下产品供一个宣传方案{product}!
{% endif %}
"""
# 创建 PromptTemplate 时指定 template_format 为 'jinja2'
prompt = PromptTemplate(
    input_variables=["手机"],
    template=template,
    template_format='jinja2'
)
# 格式化聊天提示并获取模型响应
response = chat(chat_prompt.format_prompt(
    input_language="English",
    output_language="Chinese",
    text="Who i am."
).to_messages())

print(response)
# 使用模板并传入参数
formatted_prompt = prompt.format(mood="happy")
print(formatted_prompt)

In [ ]:

from langchain.prompts import PromptTemplate
from langchain.schema import HumanMessage
# 引入第三方大模型
import os
from langchain_community.chat_models import ChatTongyi
from dotenv import load_dotenv

load_dotenv()
chat = ChatTongyi(
    model='qwen-plus',
    top_p=0.9,
    temperature=0.9,
    api_key=os.getenv("DASHSCOPE_API_KEY")
)

template = """
作为一个品牌宣传专家，请为以下品类进行宣传：
{% if category == "手机" %}
请为以下{{ brand }}手机提供一个宣传方案：{{ product }}
{% elif category == "电脑" %}
请为以下{{ brand }}电脑提供一个宣传方案：{{ product }}
{% else %}
请为以下{{ brand }}{{ category }}提供一个宣传方案：{{ product }}
{% endif %}
"""

# 创建 PromptTemplate 时指定 template_format 为 'jinja2'，并将 validate_template 设置为 False
prompt = PromptTemplate(
    input_variables=["category", "brand", "product"],
    template=template,
    template_format='jinja2',
    validate_template=False
)

# 设置品牌宣传的参数
category = "手机"
brand = "iPhone"
product = "iPhone 13 Pro Max"

# 格式化品牌宣传提示
formatted_prompt = prompt.format(
    category=category,
    brand=brand,
    product=product
)

# 将格式化后的提示包装为 HumanMessage，并传递给 chat()
response = chat([HumanMessage(content=formatted_prompt)])

print(response.content)


In [ ]:

from langchain.prompts import PromptTemplate
from langchain.schema import HumanMessage
# 引入所需的模型，例如 OpenAI 的 ChatOpenAI
import os
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv

load_dotenv()

# 初始化聊天模型
chat = ChatTongyi(
    model='qwen-plus',
    top_p=0.9,
    temperature=0.9,
    api_key=os.getenv("DASHSCOPE_API_KEY")
)

# 定义模板，使用 Jinja2 的循环结构
template = """
您好！

我们正在进行一项调查，想请您回答以下问题：

{% for question in questions %}
{{ loop.index }}. {{ question }}
{% endfor %}

请您依次回答以上问题，非常感谢您的参与！
"""

# 创建 PromptTemplate，指定 template_format为'jinja2'，并将 validate_template 设置为 False
prompt = PromptTemplate(
    input_variables=["questions"],
    template=template,
    template_format='jinja2',
    validate_template=False
)

# 定义问题列表
questions = [
    "您喜欢的手机是什么？",
    "您的喜欢手机的价格段位在多少？",
    "您的大概期手机的消费占一个月工资的比例是多少？",
    "您最喜欢的手机颜色是什么？",
    "您对手机改进任何建议吗？"
]

# 格式化提示词
formatted_prompt = prompt.format(questions=questions)

# 将格式化后的提示包装为 HumanMessage，并传递给 chat()
response = chat([HumanMessage(content=formatted_prompt)])

# 打印模型的回复
print(response.content)


In [ ]:
import os
from dotenv import load_dotenv
from langchain.prompts import PromptTemplate
from langchain.schema import HumanMessage
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory  # 导入内存模块
from langchain.chat_models import ChatOpenAI  # 确保导入正确的模型
import logging

# 配置日志
logging.basicConfig(level=logging.INFO, filename='report_generation.log',
                    format='%(asctime)s - %(levelname)s - %(message)s')

# 加载环境变量，例如 API 密钥
load_dotenv()

# 初始化内存
memory = ConversationBufferMemory(
    memory_key="history",
    return_messages=True
)

try:
    # 初始化聊天模型
    chat = ChatTongyi(
        model='qwen-plus',
        top_p=0.9,
        temperature=0.9,
        api_key=os.getenv("DASHSCOPE_API_KEY")
    )
    logging.info("聊天模型初始化成功。")
except Exception as e:
    logging.error(f"聊天模型初始化失败: {e}")
    raise e

# 初始化 ConversationChain
chain = ConversationChain(
    llm=chat,
    memory=memory,
    verbose=True  # 可选，启用详细日志
)

# 定义行业名称
industry = "新能源汽车"

# 定义通用提示模板
inner_template = """
请撰写关于{industry}行业的“{section_name}”部分，内容应包括：
{section_details}

请确保内容详实且结构清晰，并引用最新的数据。
"""

inner_prompt = PromptTemplate.from_template(inner_template)

# 定义每个章节的具体要求
sections = {
    "行业概述": """
- 行业定义
- 当前市场规模
- 主要驱动因素
""",
    "市场趋势": """
- 最新发展动态
- 技术创新
- 政策影响
""",
    "竞争分析": """
- 市场份额
- 主要优势
- 主要劣势
- 最新战略动向
""",
    "未来预测": """
- 未来五年的市场预测
- 潜在的市场机会和挑战
"""
}

# 创建每个章节的提示内容
prompts = {
    section: inner_prompt.format(
        industry=industry,
        section_name=section,
        section_details=details.strip()
    )
    for section, details in sections.items()
}

def generate_section(formatted_prompt: str, section_name: str) -> str:
    """
    生成报告的一个部分。

    :param formatted_prompt: 已格式化的提示内容
    :param section_name: 报告部分名称
    :return: 模型生成的内容
    """
    try:
        logging.info(f"生成部分: {section_name}")
        logging.debug(f"提示 '{section_name}': {formatted_prompt}")
        
        # 使用 ConversationChain 生成响应
        generated_content = chain.run(formatted_prompt).strip()
        logging.info(f"部分 '{section_name}' 生成成功。")
        
        return generated_content
    except Exception as e:
        logging.error(f"生成部分 '{section_name}' 时出错: {e}")
        return f"生成部分 '{section_name}' 时出错: {e}"

def main():
    report_sections = {}
    for section, prompt in prompts.items():
        report_sections[section] = generate_section(prompt, section)

    # 组合完整报告
    complete_report = f"""
**市场分析报告：{industry}行业**

1. **行业概述**
   {report_sections.get("行业概述", "内容未生成。")}

2. **市场趋势**
   {report_sections.get("市场趋势", "内容未生成。")}

3. **竞争分析**
   {report_sections.get("竞争分析", "内容未生成。")}

4. **未来预测**
   {report_sections.get("未来预测", "内容未生成。")}

请确保所有数据均来自可靠来源，并在报告中注明数据来源。
"""

    print("市场分析报告:\n")
    print(complete_report)

    # 可选择将报告保存到文件
    try:
        with open("市场分析报告.txt", "w", encoding="utf-8") as file:
            file.write(complete_report)
        logging.info("报告已成功保存到 '市场分析报告.txt'。")
    except Exception as e:
        logging.error(f"保存报告到文件时发生错误: {e}")

if __name__ == "__main__":
    main()
